# CommonLit - XGBooost  

This notebook uses CountVectorizer and XGBooost to make predictions

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import spacy

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
test_df  = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
train_df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
# train_df

In [ ]:
# Source: https://www.kaggle.com/bryanb/first-approach-using-xgboost-with-hyperopt

sp        = spacy.load('en_core_web_sm')
stopwords = stopwords.words('english')
snowball  = SnowballStemmer(language='english')

def preprocessing_excerpt(text):
    text = text.lower()
    text = word_tokenize(text)
    text = [ x for x in text if x not in stopwords ]
    text = [ snowball.stem(x) for x in text ]
    text = " ".join(text)
    return str(sp(text))


train_df['excerpt_clean'] = train_df['excerpt'].apply(preprocessing_excerpt)
test_df[ 'excerpt_clean'] = test_df[ 'excerpt'].apply(preprocessing_excerpt)
train_df

In [ ]:
# vectorizer       = TfidfVectorizer()
vectorizer         = CountVectorizer()
vectorizer.fit(np.concatenate([ train_df['excerpt_clean'].values, test_df['excerpt_clean'].values ]))
train_vectors      = vectorizer.transform(train_df['excerpt_clean'].values)
test_vectors       = vectorizer.transform( test_df['excerpt_clean'].values)
# feature_names      = vectorizer.get_feature_names()
# test_feature_names = vectorizer.get_feature_names()

X_train = pd.DataFrame(train_vectors.todense())
y_train = train_df['target'].values
X_test  = pd.DataFrame(test_vectors.todense()) 

In [ ]:
regressor = xgb.XGBRegressor()
regressor.fit(X_train, y_train)
y_pred    = regressor.predict(X_test)    
y_pred

In [ ]:
submission_df = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submission_df['target'] = y_pred
submission_df.to_csv('submission.csv', index=False)
submission_df